In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow.parquet as pq

In [3]:
data = pd.read_parquet("../fhvhv_tripdata_2021-01.parquet", engine="pyarrow")
print(data.head())

  hvfhs_license_num dispatching_base_num originating_base_num  \
0            HV0003               B02682               B02682   
1            HV0003               B02682               B02682   
2            HV0003               B02764               B02764   
3            HV0003               B02764               B02764   
4            HV0003               B02764               B02764   

     request_datetime   on_scene_datetime     pickup_datetime  \
0 2021-01-01 00:28:09 2021-01-01 00:31:42 2021-01-01 00:33:44   
1 2021-01-01 00:45:56 2021-01-01 00:55:19 2021-01-01 00:55:19   
2 2021-01-01 00:21:15 2021-01-01 00:22:41 2021-01-01 00:23:56   
3 2021-01-01 00:39:12 2021-01-01 00:42:37 2021-01-01 00:42:51   
4 2021-01-01 00:46:11 2021-01-01 00:47:17 2021-01-01 00:48:14   

     dropoff_datetime  PULocationID  DOLocationID  trip_miles  ...  sales_tax  \
0 2021-01-01 00:49:07           230           166        5.26  ...       1.98   
1 2021-01-01 01:18:21           152           167       

In [4]:
data['driver_pay_per_mile'] = data['driver_pay'] / data['trip_miles']
data['trip_duration'] = (data['dropoff_datetime'] - data['pickup_datetime']).dt.total_seconds()/60
data['driver_pay_per_minute'] = data['driver_pay'] / data['trip_duration']

print(data.head())

  hvfhs_license_num dispatching_base_num originating_base_num  \
0            HV0003               B02682               B02682   
1            HV0003               B02682               B02682   
2            HV0003               B02764               B02764   
3            HV0003               B02764               B02764   
4            HV0003               B02764               B02764   

     request_datetime   on_scene_datetime     pickup_datetime  \
0 2021-01-01 00:28:09 2021-01-01 00:31:42 2021-01-01 00:33:44   
1 2021-01-01 00:45:56 2021-01-01 00:55:19 2021-01-01 00:55:19   
2 2021-01-01 00:21:15 2021-01-01 00:22:41 2021-01-01 00:23:56   
3 2021-01-01 00:39:12 2021-01-01 00:42:37 2021-01-01 00:42:51   
4 2021-01-01 00:46:11 2021-01-01 00:47:17 2021-01-01 00:48:14   

     dropoff_datetime  PULocationID  DOLocationID  trip_miles  ...  tips  \
0 2021-01-01 00:49:07           230           166        5.26  ...  0.00   
1 2021-01-01 01:18:21           152           167        3.65  ...

In [5]:
import tensorflow as tf
from tensorflow import keras

In [11]:
# create a neural network model for predicting driver pay per mile based on all trip data. Create a model with 3 hidden layers, each with 64 nodes and ReLU activation functions. Use the Adam optimizer and mean squared error loss function. Train the model for 10 epochs.

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(2,)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='linear')
])

# training data
training_data = data[1:data.shape[0]//2]
target_data = data[data.shape[0]//2:]
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(training_data[['trip_miles', 'trip_duration']], training_data['driver_pay'], epochs=5)

# Use the model to predict driver pay per mile for the first 5 trips in the dataset. Compare the predicted values to the actual values.

predictions = model.predict(target_data[['trip_miles', 'trip_duration']] )
print(predictions[:5])
print(target_data['driver_pay'][:5])



c:\Users\bhavy\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
186070/186070 ━━━━━━━━━━━━━━━━━━━━ 234s 1ms/step - loss: 10.3158
Epoch 2/5
186070/186070 ━━━━━━━━━━━━━━━━━━━━ 220s 1ms/step - loss: 9.3862
Epoch 3/5
186070/186070 ━━━━━━━━━━━━━━━━━━━━ 219s 1ms/step - loss: 9.6645
Epoch 4/5
186070/186070 ━━━━━━━━━━━━━━━━━━━━ 221s 1ms/step - loss: 9.7176
Epoch 5/5
186070/186070 ━━━━━━━━━━━━━━━━━━━━ 227s 1ms/step - loss: 9.6589
186070/186070 ━━━━━━━━━━━━━━━━━━━━ 180s 967us/step
[[11.340802]
 [ 6.036688]
 [10.085955]
 [ 5.905987]
 [ 6.07553 ]]
5954234    10.72
5954235     5.40
5954236     9.47
5954237     5.40
5954238     6.32
Name: driver_pay, dtype: float64


In [12]:
print(model.evaluate(target_data[['trip_miles', 'trip_duration']], target_data['driver_pay']))

186070/186070 ━━━━━━━━━━━━━━━━━━━━ 189s 1ms/step - loss: 8.5704
9.355286598205566


In [13]:
model.save('my_model.keras')

In [23]:
new_model = tf.keras.models.load_model('my_model.keras')

print(new_model.predict(target_data[1:50][['trip_miles', 'trip_duration']]))
print(target_data[1:50]['driver_pay'])

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/stepWARNING:tensorflow:5 out of the last 186073 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001F8464AEAC0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
[[ 6.036688 ]
 [10.085955 ]
 [ 5.905987 ]
 [ 6.07553  ]
 [11.788852 ]
 [ 6.6538496]
 [ 6.6538496]
 [12.050205 ]
 [14.300811 ]
 [ 6.1049414]
 [ 5.953967 ]
 [ 6.961256 ]
 [49.10934  ]
 [27.036638 